# Strange behaviour of RoughPy

In [1]:
import roughpy as rp
import numpy as np

### Using integer arrays

There appears to be an issue when an integer array is passed in. The data  is intepreted in the 'wrong' way. If we use [[3, 7, 0], [0, 0, 1]] is treated as 3 increments on a 2D stream. If instead we use [[3., 7., 0.], [0., 0., 1.]], then this is treated as 2 increments on a 3D stream.

In [2]:
stream = rp.LieIncrementStream.from_increments(np.array([[3, 7, 0], [0, 0, 1]]), depth=2, coeffs=rp.Rational) 
sig = stream.signature(depth=1)
print(sig)

{ 1() 10(1) 1(2) }


Using the width argument does not appear to make a difference

In [3]:
stream = rp.LieIncrementStream.from_increments(np.array([[3, 7, 0], [0, 0, 1]]), width=3, depth=2, coeffs=rp.Rational) 
sig = stream.signature(depth=1)
print(sig)

{ 1() 10(1) 1(2) }


This means we cannot pass information on area this way, the following is still treated as a 2D stream

In [4]:
stream = rp.LieIncrementStream.from_increments(np.array([[3, 7, 0, 4], [0, 0, 1, 5]]), width=2, depth=2, coeffs=rp.Rational) 
sig = stream.signature(depth=1)
print(sig)

{ 1() 14(1) 6(2) }


This changes if we pass in a third increment, it now functions as expected

In [5]:
stream = rp.LieIncrementStream.from_increments(np.array([[3, 0, 0], [7, 0, 0], [0, 0, 1]]), width=3, depth=2, coeffs=rp.Rational) 
sig = stream.signature(depth=2)
print(sig)

{ 1() 10(1) 1(3) 50(1,1) 10(1,3) 0.5(3,3) }


Alternatively, if we pass in floats, then there is no such issue

In [6]:
stream = rp.LieIncrementStream.from_increments(np.array([[3., 7., 0.], [0., 0., 1.]]), depth=2, coeffs=rp.Rational) 
sig = stream.signature(depth=1)
print(sig)

{ 1() 3(1) 7(2) 1(3) }


### The coefficients argument

Interestingly the coefficients appear to be floats instead of rationals if we do not pass coeffs in as part of the context ctx

In [7]:
word_stream = rp.LieIncrementStream.from_increments(np.array([[3, 7, 0], [0, 0, 1]]), resolution=2, width=3, depth=2, coeffs=rp.Rational) 
sig = word_stream.signature(depth=2)
print(sig)

{ 1() 10(1) 1(2) 50(1,1) 10(1,2) 0.5(2,2) }


In [8]:
CTX = rp.get_context(width=3, depth=2, coeffs=rp.Rational)
word_stream = rp.LieIncrementStream.from_increments(np.array([[3, 7, 0], [0, 0, 1]]), resolution=2, ctx=CTX)
sig = word_stream.signature(depth=2)
print(sig)

{ 1() 10(1) 1(2) 50(1,1) 10(1,2) 1/2(2,2) }


### Using ctx has issues with two increments even if they are floats

In [9]:
stream = rp.LieIncrementStream.from_increments(np.array([[3., 7., 0.], [0., 0., 1.]]), depth=2, ctx=CTX) 
sig = stream.signature(depth=1)
print(sig)

{ 1() 10(1) 1(2) }


Feeding in three increments is as expected

In [10]:
stream = rp.LieIncrementStream.from_increments(np.array([[3., 0., 0.], [0., 7., 0.], [0., 0., 1.]]), depth=2, ctx=CTX) 
sig = stream.signature(depth=1)
print(sig)

{ 1() 3(1) 7(2) 1(3) }


### Using ctx with one increment, regardless of the width, appears to be treating increments incorrectly

In [11]:
test_array = np.zeros([1, 20])
test_array[0][4] = 1

In [12]:
test_array

array([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.]])

In [13]:
CTX = rp.get_context(width=20, depth=2, coeffs=rp.Rational)
stream = rp.LieIncrementStream.from_increments(test_array, resolution=2, ctx=CTX)
sig = stream.signature(depth=2)
print(sig)

{ 1() 1(1) 1/2(1,1) }


The above appears to be constructing a one dimensional array instead of something of width 20. If we do not use ctx, the outcome is as expected.

In [14]:
stream = rp.LieIncrementStream.from_increments(test_array, resolution=2, width=20, depth=2, coeffs=rp.Rational) 
sig = stream.signature(depth=2)
print(sig)

{ 1() 1(5) 0.5(5,5) }
